In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint

# Import API key
from config import weather_api_key
from config import snow_api_key

In [2]:
resort_data = pd.read_csv("statis/data/resorts.csv")
resort_df = pd.DataFrame(resort_data)
resort_df.head()

,Unnamed: 0,name,link,price,closest_town,region,total_len,easy_len,intermediate_len,difficult_len
0,0,Vail,https://www.skiresort.info/ski-resort/vail/,219.0,Vail,Colorado,234.0,57.0,84.0,93.0
1,1,Telluride,https://www.skiresort.info/ski-resort/telluride/,169.0,Mountain Village,Colorado,88.2,6.7,51.7,29.8
2,2,Beaver Creek,https://www.skiresort.info/ski-resort/beaver-c...,218.0,Beaver Creek Village,Colorado,150.0,28.5,64.5,57.0
3,3,Mammoth Mountain,https://www.skiresort.info/ski-resort/mammoth-...,209.0,Mammoth Lakes,Mammoth Lakes,89.8,18.4,48.5,22.9
4,4,Snowmass,https://www.skiresort.info/ski-resort/snowmass/,199.0,Snowmass Village,Aspen Snowmass,237.0,12.0,114.0,111.0


In [3]:
country = "US"

In [4]:
# Set index to name
# resort = resort_df.set_index('name')
updated_resort = resort_df.drop(['Unnamed: 0'], axis = 1)
updated_resort['country'] = country
updated_resort.tail(60)

,name,link,price,closest_town,region,total_len,easy_len,intermediate_len,difficult_len,country
463,Aldis Hill Park – St. Albans,https://www.skiresort.info/ski-resort/aldis-hi...,NaN,NaN,Vermont,0.4,0.2,0.2,0.0,US
464,Ruby Mountain,https://www.skiresort.info/ski-resort/ruby-mou...,NaN,Lamoille,NaN,NaN,NaN,NaN,NaN,US
465,North Cascade Heliskiing – Mazama,https://www.skiresort.info/ski-resort/north-ca...,NaN,Mazama,NaN,NaN,NaN,NaN,NaN,US
466,Powderhouse Hill – South Berwick,https://www.skiresort.info/ski-resort/powderho...,5.0,South Berwick,Maine,0.5,0.5,0.0,0.0,US
467,Keyes Peak – Florence,https://www.skiresort.info/ski-resort/keyes-pe...,12.0,Florence,Wisconsin,0.5,0.5,0.0,0.0,US
468,Salmonberry – Valdez,https://www.skiresort.info/ski-resort/salmonbe...,NaN,Valdez,Alaska,0.4,0.4,0.0,0.0,US
469,Rocking Horse Ranch,https://www.skiresort.info/ski-resort/rocking-...,NaN,Highland,The Catskills,0.3,0.3,0.0,0.0,US
470,Great Northern Powder Guides,https://www.skiresort.info/ski-resort/great-no...,NaN,Whitefish,read more,NaN,NaN,NaN,NaN,US
471,Cascade Powder Guides,https://www.skiresort.info/ski-resort/cascade-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US
472,Petoskey Winter Sports Park,https://www.skiresort.info/ski-resort/petoskey...,NaN,Petoskey,Michigan,0.2,0.2,0.0,0.0,US


In [5]:
len(updated_resort)

523

In [6]:
# resort_names  = list(updated_resort_list['name'])
# # # # resort_names.head()

In [7]:
# cities = list(updated_resort['closest_town'])
# # # cities

In [8]:
# Ran into issues with nan values in the cities list, so need to remove those. -- NVM DON'T USE THIS-- use dropna
# Drop na will remove entire row where an na is present in the dataframe
# updated_resort['closest_town'].isnull().values.any() -- this was true so use the code below to remove nan
# updated_resort_list = updated_resort.replace(np.nan, 'NA', regex=True)
# updated_resort_test.head()
# updated_resort_list['closest_town'].isnull().values.any()
# updated_resort.isnull().values.any()
# updated_resort_df = updated_resort.replace(np.nan, 'NA', regex=True)
# updated_resort_df.isnull().values.any()

In [9]:
# # resorts_no_na = updated_resort.dropna()
# # len(resorts_no_na)
# if cities == 'nan':
#     updated_resort.dropna()

In [10]:
# updated_resort.dtypes

In [11]:
test_df = updated_resort[updated_resort['closest_town'].notna()]

In [12]:
len(test_df)

470

In [13]:
cities = list(test_df['closest_town'])
# # cities

In [14]:
resort_names = list(test_df['name'])

In [15]:
# len(cities)
# len(resort_names)

In [18]:
# "api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}"
# Lists to call
new_city = []
date = [] # Make this date updated
lat = []
long = []
temp_main = [] # Use this to get current temperature
feels_like = []
temp_min = []
temp_max = []
temp_description = []
hum = []
wind_speed = []
cloudiness = []

x = 0 

print('Beginning Data Retrieval')
print("-"*64)  
for city in cities:
    url = "http://api.openweathermap.org/data/2.5/weather?"
    units = 'metric'
#     query_url = f'q={resort}&appid={weather_api_key}'
    query_url = f'{url}appid={weather_api_key}&q={city}&units={units}'
    
    
    try:
        # Get metrics from api doc/response
        response = requests.get(query_url + city).json()
        x = x + 1
        print(f'Processing Record {x} | {city}')
#         resort_names.append(response['name'])
        new_city.append(response['name'])
        date.append((datetime.fromtimestamp(response['dt'])).strftime('%m/%d/%y'))
        lat.append(response['coord']['lat'])
        long.append(response['coord']['lon'])
        temp_main.append(response['main']['temp'])
        feels_like.append(response['main']['feels_like'])
        temp_min.append(response['main']['temp_min'])
        temp_max.append(response['main']['temp_max'])
        temp_description.append(response['weather'][0]['description'])       
        hum.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        cloudiness.append(response['clouds']['all'])
    except KeyError:
        print("City not found", city)

Beginning Data Retrieval
----------------------------------------------------------------
Processing Record 1 | Vail
Processing Record 2 | Mountain Village
Processing Record 3 | Beaver Creek Village
City not found Beaver Creek Village
Processing Record 4 | Mammoth Lakes
Processing Record 5 | Snowmass Village
Processing Record 6 | Breckenridge
Processing Record 7 | Keystone
Processing Record 8 | Killington
Processing Record 9 | Sunday River
City not found Sunday River
Processing Record 10 | Heavenly Village
City not found Heavenly Village
Processing Record 11 | Winter Park
Processing Record 12 | Stowe
Processing Record 13 | Olympic Valley
City not found Olympic Valley
Processing Record 14 | Steamboat Springs
Processing Record 15 | Park City
Processing Record 16 | Big Sky Mountain Village
Processing Record 17 | Highlands Village
City not found Highlands Village
Processing Record 18 | Frisco
Processing Record 19 | Alpine Meadows
Processing Record 20 | Alta
Processing Record 21 | Aspen
Pro

Processing Record 227 | La Crosse
Processing Record 228 | Oroville
Processing Record 229 | Summerhaven
Processing Record 230 | Jennerstown
Processing Record 231 | Norway
Processing Record 232 | Rumford
Processing Record 233 | Elkhorn
Processing Record 234 | Iron River
Processing Record 235 | Waynesville
Processing Record 236 | Las Vegas
Processing Record 237 | Casper
Processing Record 238 | Old Forge
Processing Record 239 | Middlefield
Processing Record 240 | Camden
Processing Record 241 | Southington
Processing Record 242 | Mansfield
Processing Record 243 | Cordova
Processing Record 244 | Mars Hill
Processing Record 245 | White Lake
Processing Record 246 | Naples
Processing Record 247 | Paoli
Processing Record 248 | Zanesfield
Processing Record 249 | Thompsonville
Processing Record 250 | Westford
Processing Record 251 | Keene
Processing Record 252 | Harrisonburg
Processing Record 253 | East Madison
Processing Record 254 | Clarkston
Processing Record 255 | Los Alamos
Processing Record 

Processing Record 467 | Eagle
Processing Record 468 | Winter Park
Processing Record 469 | Florence
Processing Record 470 | Huntsville (Utah)
City not found Huntsville (Utah)


In [ ]:
len(cities)

In [20]:
resort_info = {
    "City": new_city,
    "Date": date,
    "Latitude": lat,
    "Longitude": long,
    "Temperature": temp_main,
    "Feels_like":  feels_like,
    "Temp_min": temp_min,
    "Temp_max": temp_max,
    "Temp_description": temp_description,
    "Humidity": hum,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed
}
resort_info_df = pd.DataFrame(resort_info)
resort_info_df.head()

,City,Date,Latitude,Longitude,Temperature,Feels_like,Temp_min,Temp_max,Temp_description,Humidity,Cloudiness,Wind Speed
0,Vail,02/20/21,32.0479,-110.7120,294.58,289.15,293.15,295.93,clear sky,9,0,3.13
1,Mountain Village,02/20/21,37.9314,-107.8565,276.98,268.42,276.15,278.15,clear sky,32,1,7.72
2,Mammoth Lakes,02/20/21,37.6485,-118.9721,271.99,263.33,271.48,272.59,scattered clouds,40,40,7.72
3,Snowmass Village,02/20/21,39.2130,-106.9378,275.52,267.51,272.04,278.15,clear sky,28,1,6.69
4,Breckenridge,02/20/21,32.7557,-98.9023,286.99,280.55,285.15,288.15,clear sky,36,1,6.17


In [22]:
# resort_info_df.to_csv('statis/data/city_weather.csv'index=False)

In [23]:
len(resort_info_df)

449

In [ ]:
# # Test
# resort_name = ["Davis", "Chicago"]

In [25]:
# SNOW API
daily_chance_snow = []
hourly_chance_snow = []
local_time = []
hour = []

x = 0 
print('Beginning Data Retrieval')
print("-"*64)  
for city in new_city:
#     url = "http://api.openweathermap.org/data/2.5/weather?"
    url = "http://api.weatherapi.com/v1/forecast.json?"
#     units = 'metric'
    days = "1"
    query_url = f'{url}key={snow_api_key}&q={city}&days={days}'
    
    
    try:
        # Get metrics from api doc/response
        response = requests.get(query_url + city).json()
        x = x + 1
        print(f'Processing Record {x} | {city}')
        daily_chance_snow.append(response['forecast']['forecastday'][0]['day']['daily_chance_of_snow'])
        hourly_chance_snow.append(response['forecast']['forecastday'][0]['hour'][0]['chance_of_snow'])
        local_time.append(response['location']['localtime'])
        hour.append(response['forecast']['forecastday'][0]['hour'][0]['time'])
    except KeyError:
        print("City not found", city)

Beginning Data Retrieval
----------------------------------------------------------------
Processing Record 1 | Vail
Processing Record 2 | Mountain Village
Processing Record 3 | Mammoth Lakes
Processing Record 4 | Snowmass Village
Processing Record 5 | Breckenridge
Processing Record 6 | Keystone
Processing Record 7 | Killington
Processing Record 8 | Winter Park
Processing Record 9 | Stowe
Processing Record 10 | Steamboat Springs
Processing Record 11 | Park City
Processing Record 12 | Big Sky Mountain Village
Processing Record 13 | Frisco
Processing Record 14 | Alpine Meadows
Processing Record 15 | Alta
Processing Record 16 | Aspen
Processing Record 17 | Teton Village
Processing Record 18 | Park City
Processing Record 19 | Mount Crested Butte
Processing Record 20 | Eden
Processing Record 21 | Lake Placid
Processing Record 22 | Bend
Processing Record 23 | Truckee
Processing Record 24 | Sandpoint
Processing Record 25 | Keystone
Processing Record 26 | Government Camp
Processing Record 27 |

Processing Record 243 | Eureka
Processing Record 244 | Brighton
Processing Record 245 | Warwick
Processing Record 246 | Wild Rose
Processing Record 247 | Boone
Processing Record 248 | Gatlinburg
Processing Record 249 | Wilmot
Processing Record 250 | Fairbanks
Processing Record 251 | Bellaire
Processing Record 252 | Anchorage
Processing Record 253 | Andalusia
Processing Record 254 | Farmington
Processing Record 255 | Salmo
Processing Record 256 | Weston
Processing Record 257 | Sheridan
Processing Record 258 | Taylors Falls
Processing Record 259 | Kimball
Processing Record 260 | Gaylord
Processing Record 261 | Auburn
Processing Record 262 | Sioux Falls
Processing Record 263 | Hanover
Processing Record 264 | Manchester
Processing Record 265 | Elko
Processing Record 266 | Slinger
Processing Record 267 | Duluth
Processing Record 268 | Kalispell
Processing Record 269 | Otis
Processing Record 270 | Libby
Processing Record 271 | Walhalla
Processing Record 272 | Morris
Processing Record 273 | S

In [ ]:
# local_time

In [ ]:
# response

In [27]:
test = {
    "Resort": new_city,
    "date": date, 
#     "hour": hour,
    "daily_chance_snow": daily_chance_snow,
    "hourly_chance_snow": hourly_chance_snow,
}
test_df = pd.DataFrame(test)
test_df.head()

,Resort,date,daily_chance_snow,hourly_chance_snow
0,Vail,02/20/21,0,0
1,Mountain Village,02/20/21,12,0
2,Mammoth Lakes,02/20/21,42,32
3,Snowmass Village,02/20/21,23,0
4,Breckenridge,02/20/21,0,0


In [ ]:
combined = {
    "City": new_city,
    "Date": date,
    "Latitude": lat,
    "Longitude": long,
    "Temperature": temp_main,
    "Feels_like":  feels_like,
    "Temp_min": temp_min,
    "Temp_max": temp_max,
    "Temp_description": temp_description,
    "Humidity": hum,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed
}
combied_df = pd.DataFrame(combined)
combied_df.head()

In [ ]:
# resort_info_df.to_csv('statis/data/city_weather.csv'index=False)